# Setting Up the Deployment/Test Environment
In this section, we will be creating environmental variables and saving them in a .env file in the **common** folder. These variables will be used in later sections.

## Set Up Global Variables

First, change the azureSubscriptionId, resourceLocation, and resourceGroupName variables. The resourceGroupName must be less than 16 characters in length, with only lowercase letters and numbers. It defaults to a random unique name using the cell below. 

### Note:
Please read about naming conventions and restrictions before continuing:  
- [Naming rules and restrictions for Azure resources](https://docs.microsoft.com/en-us/azure/azure-resource-manager/management/resource-name-rules)  
- [Create a unique name for a resource](https://docs.microsoft.com/en-us/azure/azure-resource-manager/templates/template-functions-string#uniquestring)  

In [ ]:
# First, we will create a random name to append our Azure resource names. The name is not guaranteed to be unique, but doing this will increase the chances of generating a unique name. If the code snippet returns an error such as "resource name already exist", you should re-generate new name values.
import uuid 
nearlyUniqueName = uuid.uuid4().hex[:3].lower()

In [ ]:
# Mandatory settings section
# ---------------------------

# Define the ID of the Azure subscription that will be used
# You can find your subscription ID at https://account.azure.com/subscriptions
azureSubscriptionId = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"

# Define the datacenter location of the Azure services that will be created.This part assumes that your chosen resourceLocation has enough GPU (N Series VM's) quota. Please double-check whether the region you've chosen has GPU Quota for Standard NC6s v2 or not. If it doesn't, please choose another location or GPU VM Type.
#Please select a region to deploy resources from this list: eastus2, northcentralus, uksouth, westus2.
resourceLocation = "westus2" 

# Define the name of the resource group you will use to contain all of the Azure services that will be created.
# Please use a globally unique resource group name
resourceGroupName = "evslab" + nearlyUniqueName
# resourceGroupName = "<RESOURCE_GROUP_NAME>"  # Uncomment this command if you want to create your own Resource Group Name

#Parameters for Azure Kubernetes cluster

#Node resource group name for AKS 
noderesourceGroupName = resourceGroupName+'evsnoderg'
aksclustername = resourceGroupName+'evsakscluster'
aksdns = resourceGroupName+'aksdns'
gpuvmtype = 'Standard_NC6s_v2'

#Parameters for EVS Deployment

videoinputfolderondevice="/evs/input"
evsclientimage ="evsclient:1.0"
evsclientendpointlocal="endpointLocal=http://rocket:7788/api/ImageItems"
evsclientendpointtype="endpointType=rocket"
#evsclientendpointremote="endpointRemote=http://<Update with your AKS DNS name eg: evsfoaks1.westus2.cloudapp.azure.com >:8888/api/Video" #We will update this value later in this section
evsclientendpointremote = 'endpointRemote='+'http://'+aksdns+'.'+resourceLocation+'.cloudapp.azure.com:8888/api/Video'
evsclientfolderondevice="/evs/evsclient"
evsclientline="line=https://aka.ms/linesample"
evsclientvideourl="videoUrl=sample.mp4"
evsclientsamplingrate="ingestSamplingRate=3"
evsclientcategory="cat=car,bicycle"
rocketoutputfolderondevice = "/evs/rocket"
rocketimage = "rocket:1.0"
rocketpipeline="pipeline=5"
rocketline="line=https://aka.ms/linesample"

# Optional settings section
# ---------------------------

# Azure IoT Hub service
iotHubServiceName = resourceGroupName+'iothub'

# Define a unique IoT Edge device ID
iotDeviceId = resourceGroupName+'iotdevid'

# Azure Storage services
storageServiceName = resourceGroupName+'storage'


In [ ]:
import re
pattern = '^[a-z0-9_]{1,16}$'
if not re.search(pattern, resourceGroupName): 
      print("The resourceGroupName must be less than 16 characters in length, with only lowercase letters and numbers.")

## Check Azure CLI Version
Azure CLI is required to run this sample. Please check if Azure CLI has been installed on your machine. If it has not, then visit the [requirements for running LVA on Jupyter](requirements.md) page.

In [ ]:
!az --version

If the command does not run successfully, try reopening VSCode or restarting your development PC.

## Login to Azure
> <span style="color:red; font-weight:bold"> [!IMPORTANT] </span>  
> Run the following command and **follow the instructions** in the corresponding output cell (i.e., a browser popup). You can run the `az logout` command or stop the kernel to logout.

In [ ]:
!az login -o None

## Set Active Azure Subscription
In case you have more than one Azure subscription, we will set the one you specified in the **azureSubscriptionId** variable as the default to be used.

In [ ]:
!az account set --subscription $azureSubscriptionId

## Save Global Variables in .env File
Check if a [.env](.env) file already exists. If not, the following cell will create a new one. The .env file is where we will store our variables to share with other notebooks of this sample.

After completing the rest of these notebooks, your .env file will look something like this ("..." to indicate user-specific information):

```
NEARLY_UNIQUE_NAME='...'
SUBSCRIPTION_ID='...'
RESOURCE_LOCATION='...'
RESOURCE_GROUP='...'
IOT_HUB_SERVICE_NAME='...'
IOT_DEVICE_ID='...'
STORAGE_SERVICE_NAME='...'
NODE_RESOURCEGROUP='...'
AKS_CLUSTERNAME='...'
AKS_DNS='...'
GPU_VM_SIZE='...'
VIDEO_INPUT_FOLDER_ON_DEVICE='...'
EVSCLIENT_IMAGE='...'
EVSCLIENT_ENDPOINTLOCAL='...'
EVSCLIENT_ENDPOINTTYPE='...'
EVSCLIENT_ENDPOINTREMOTE='...'
EVSCLIENT_OUTPUT_FOLDER_ON_DEVICE='...'
EVSCLIENT_LINE='...'
EVSCLIENT_VIDEOURL='...'
EVSCLIENT_SAMPLINGRATE='...'
EVSCLIENT_CATEGORY='...'
ROCKET_OUTPUT_FOLDER_ON_DEVICE='...'
ROCKET_IMAGE='...'
ROCKET_PIPELINE='...'
ROCKET_LINE='...'
```

In [ ]:
from pathlib import Path
from dotenv import set_key, get_key, find_dotenv

envPath = find_dotenv()
if envPath=="":
    Path(".env").touch()
    envPath = find_dotenv()
    
print(".env file path: {0}".format(envPath))

In [ ]:
set_key(envPath, "NEARLY_UNIQUE_NAME", nearlyUniqueName)
set_key(envPath, "SUBSCRIPTION_ID", azureSubscriptionId)
set_key(envPath, "RESOURCE_LOCATION", resourceLocation)
set_key(envPath, "RESOURCE_GROUP", resourceGroupName)
set_key(envPath, "IOT_HUB_SERVICE_NAME", iotHubServiceName)
set_key(envPath, "IOT_DEVICE_ID", iotDeviceId)
set_key(envPath, "STORAGE_SERVICE_NAME", storageServiceName)
set_key(envPath, "NODE_RESOURCEGROUP", noderesourceGroupName)
set_key(envPath, "AKS_CLUSTERNAME", aksclustername)
set_key(envPath, "AKS_DNS", aksdns)
set_key(envPath, "GPU_VM_SIZE", gpuvmtype)
set_key(envPath, "VIDEO_INPUT_FOLDER_ON_DEVICE", videoinputfolderondevice)
set_key(envPath, "EVSCLIENT_IMAGE", evsclientimage)
set_key(envPath, "EVSCLIENT_ENDPOINTLOCAL", evsclientendpointlocal)
set_key(envPath, "EVSCLIENT_ENDPOINTTYPE", evsclientendpointtype)
set_key(envPath, "EVSCLIENT_ENDPOINTREMOTE", evsclientendpointremote)
set_key(envPath, "EVSCLIENT_OUTPUT_FOLDER_ON_DEVICE", evsclientfolderondevice)
set_key(envPath, "EVSCLIENT_LINE", evsclientline)
set_key(envPath, "EVSCLIENT_VIDEOURL", evsclientvideourl)
set_key(envPath, "EVSCLIENT_SAMPLINGRATE", evsclientsamplingrate)
set_key(envPath, "EVSCLIENT_CATEGORY", evsclientcategory)
set_key(envPath, "ROCKET_OUTPUT_FOLDER_ON_DEVICE", rocketoutputfolderondevice)
set_key(envPath, "ROCKET_IMAGE", rocketimage)
set_key(envPath, "ROCKET_PIPELINE", rocketpipeline)
tempVar = set_key(envPath, "ROCKET_LINE", rocketline)

## Validation Steps
If all the code cells above have successfully finished running, check the [.env file](.env) to see if all the variables set above have been included. If you cannot find the .env file, try refershing the explorer pane.

<img src="../images/_refresh_explorer.JPG" width=355px/>  
 
Then, return to the Readme page (3.2) to continue.   